# Agrango DB

```aql
CREATE COLLECTION users
```

```aql
INSERT { name: "Alice", age: 30, email: "alice@example.com" } INTO users
INSERT { name: "Bob", age: 25, email: "bob@example.com" } INTO users
INSERT { name: "Charlie", age: 28, email: "charlie@example.com" } INTO users
```

```aql
FOR u IN users RETURN u
```

In [1]:
%%capture

!pip install python-arango

### 기본 실행

In [2]:
from arango import ArangoClient

# 1. ArangoDB 연결
client = ArangoClient()
db = client.db(
    name="_system",                 # 기본 DB
    username="root",               # ArangoDB 사용자명
    password="openSesame"  # 비밀번호 (본인의 설정에 맞게 입력)
)

# 2. 새 컬렉션 생성 (이미 존재하면 생략)
if not db.has_collection("users"):
    users = db.create_collection("users")
else:
    users = db.collection("users")

# 3. 예제 문서 삽입
users.insert({"_key": "alice", "name": "Alice", "age": 30})
users.insert({"_key": "bob", "name": "Bob", "age": 25})

# 4. 문서 전체 조회
for doc in users.all():
    print(doc)

{'_key': 'alice', '_id': 'users/alice', '_rev': '_k_KOyJ----', 'name': 'Alice', 'age': 30}
{'_key': 'bob', '_id': 'users/bob', '_rev': '_k_KOyJG---', 'name': 'Bob', 'age': 25}


In [3]:
# 1. 엣지 컬렉션 생성
if not db.has_collection("friends"):
    friends = db.create_collection("friends", edge=True)
else:
    friends = db.collection("friends")

# 2. 엣지 문서 삽입
friends.insert({
    "_from": "users/alice",
    "_to": "users/bob"
})

{'_id': 'friends/909', '_key': '909', '_rev': '_k_KPXxa---'}

In [4]:
from arango import ArangoClient

# 1. ArangoDB 연결
client = ArangoClient()
db = client.db("_system", username="root", password="openSesame")

# 2. Vertex 컬렉션 생성 (people)
if not db.has_collection("people"):
    db.create_collection("people")

# 3. Edge 컬렉션 생성 (knows)
if not db.has_collection("knows"):
    db.create_collection("knows", edge=True)

# 4. Named Graph가 이미 있는지 확인하고 없으면 생성
if not db.has_graph("social_graph"):
    graph = db.create_graph("social_graph")

    # 엣지 정의: knows 컬렉션이 people → people 관계를 가짐
    graph.create_edge_definition(
        edge_collection="knows",
        from_vertex_collections=["people"],
        to_vertex_collections=["people"]
    )
else:
    graph = db.graph("social_graph")

# 5. 정점 추가
people = db.collection("people")
people.insert({"_key": "alice", "name": "Alice"})
people.insert({"_key": "bob", "name": "Bob"})
people.insert({"_key": "charlie", "name": "Charlie"})

# 6. 엣지 추가
knows = db.collection("knows")
knows.insert({"_from": "people/alice", "_to": "people/bob"})
knows.insert({"_from": "people/bob", "_to": "people/charlie"})


{'_id': 'knows/1057', '_key': '1057', '_rev': '_k_KRfIG---'}

### AQL로 그래프 탐색 

| 항목          | 설명                 |
| ----------- | ------------------ |
| Vertex 컬렉션  | `people`           |
| Edge 컬렉션    | `knows`            |
| Named Graph | `social_graph`     |
| 탐색 방향       | `OUTBOUND` (방향 있음) |
| 탐색 범위       | `1..1` (1단계만)      |


In [5]:
query = """
FOR v, e IN 1..1 OUTBOUND 'people/alice' GRAPH 'social_graph'
RETURN v
"""
cursor = db.aql.execute(query)
for doc in cursor:
    print(doc)

{'_key': 'bob', '_id': 'people/bob', '_rev': '_k_KRfHq---', 'name': 'Bob'}
